In [2]:
import pandas as pd

# Carregar dados da camada Silver

df = pd.read_csv('c:/Users/rafae/PIPELINE/projeto_pipeline/silver/twitter_sentment.csv')

AGREGAÇÃO 1: Métricas Diárias

In [4]:
metricas_diarias = df.groupby('data').agg({
    'indice': 'count',          # Total de tweets por dia
    'usuario': 'nunique',       # Usuários únicos por dia
}).reset_index()

metricas_diarias.columns = ['data', 'total_tweets', 'usuarios_unicos']

metricas_diarias.to_csv('c:/Users/rafae/PIPELINE/projeto_pipeline/gold/metricas_diarias.csv', index=False)


AGREGAÇÃO 2: Análise de Clientes

In [6]:
# Agrupar por usuário (id_cliente equivalente) para calcular:
# Recency (última postagem), Frequency (número de posts).

analise_usuarios = df.groupby('usuario').agg({
    'data': 'max',       # Última postagem (recency)
    'indice': 'count'    # Frequência de postagens
}).reset_index()

# Renomear colunas para clareza
analise_usuarios.columns = ['usuario', 'ultima_postagem', 'frequencia']

# Calcular dias desde a última postagem
analise_usuarios['dias_desde_ultima_postagem'] = (pd.to_datetime('today') - pd.to_datetime(analise_usuarios['ultima_postagem'])).dt.days

# Salvar resultado
analise_usuarios.to_csv('c:/Users/rafae/PIPELINE/projeto_pipeline/gold/analise_usuarios.csv', index=False)


AGREGAÇÃO 3: Identificador de Menções

In [ ]:
print(df.columns)
df = df.rename(columns={"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D" : "texto"})


Index(['indice', 'user_id', 'data', 'consulta', 'usuario', 'texto',
       'data_ingestao', 'fonte_arquivo', 'data_processamento'],
      dtype='object')
Index(['indice', 'user_id', 'data', 'consulta', 'usuario', 'texto',
       'data_ingestao', 'fonte_arquivo', 'data_processamento'],
      dtype='object')


In [ ]:

# Garantir que a coluna texto é string
df['texto'] = df['texto'].astype(str)

# Funções para identificar respostas, menções e retweets
def conta_respostas(texto):
    return int(texto.strip().startswith('@'))

def conta_mencoes(texto):
    return texto.count('@') - conta_respostas(texto)

def conta_retweets(texto):
    return int(texto.strip().upper().startswith('RT'))

# Aplicar funções usando a nova coluna 'texto'
df['resposta'] = df['texto'].apply(conta_respostas)
df['mencoes'] = df['texto'].apply(conta_mencoes)
df['retweet'] = df['texto'].apply(conta_retweets)

# Agrupando por usuário
engajamento_mencoes = df.groupby('usuario').agg({
    'resposta': 'sum',
    'mencoes': 'sum',
    'retweet': 'sum',
    'indice': 'count'
}).reset_index()

engajamento_mencoes.columns = ['usuario', 'total_respostas', 'total_mencoes', 'total_retweets', 'total_tweets']

# Salvar resultado em CSV
engajamento_mencoes.to_csv('c:/Users/rafae/PIPELINE/projeto_pipeline/gold/identificador_mencoes.csv', index=False)


In [17]:
print(df.columns)


Index(['indice', 'user_id', 'data', 'consulta', 'usuario', 'texto',
       'data_ingestao', 'fonte_arquivo', 'data_processamento', 'resposta',
       'mencoes', 'retweet'],
      dtype='object')


In [18]:
import os
print(os.getcwd())

c:\Users\rafae\PIPELINE\projeto_pipeline\gold
